In [9]:
# import numpy as np
# import cv2 as cv
# import os

# drawing = False # true if mouse is pressed
# ix,iy = -1,-1

# # mouse callback function
# def draw_rectanlge(event, x, y, flags, param):
#     """ Draw rectangle on mouse click and drag """
#     global ix,iy,drawing,mode
#     # if the left mouse button was clicked, record the starting and set the drawing flag to True
#     if event == cv.EVENT_LBUTTONDOWN:
#         drawing = True
#         ix,iy = x,y
#     # mouse is being moved, draw rectangle
#     elif event == cv.EVENT_MOUSEMOVE:
#         if drawing == True:
#             cv.rectangle(img, (ix, iy), (x, y), (255, 255, 0), 2)
#     # if the left mouse button was released, set the drawing flag to False
#     elif event == cv.EVENT_LBUTTONUP:
#         drawing = False

# # create a black image (height=360px, width=512px), a window and bind the function to window
# img = cv.imread(os.path.join(os.getcwd(), '..','img', 'lantern.jpg'))
# cv.namedWindow('image')
# cv.setMouseCallback('image',draw_rectanlge)

# while(1):
#     cv.imshow('image',img)
#     if cv.waitKey(20) & 0xFF == 27:
#         break

# cv.destroyAllWindows()

In [10]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import os
from copy import copy

In [11]:
class data_and_events:
    def __init__(self, path) -> None:
        self.start_point = (-1, -1)
        img = cv.imread(path)
        self.drawing = False
        self.img = copy(img)
        self.img_copy = copy(img)

    def mouse_callback(self, event, x, y, flags, param):
        if event == cv.EVENT_LBUTTONDOWN:
            self.drawing = True
            self.start_point = (x, y)
        elif event == cv.EVENT_MOUSEMOVE:
            if self.drawing:
                self.img = copy(self.img_copy)
                cv.rectangle(self.img, self.start_point, (x, y), (0, 255, 0), 2)
        elif event == cv.EVENT_LBUTTONUP:
            self.drawing = False
            cv.rectangle(self.img, self.start_point, (x, y), (0, 255, 0), 2)

    def get_image(self):
        return self.img

    def keyboard_handler(self):
        key_press = cv.waitKey(1) & 0xFF
        if key_press == 27:  # esc key
            return -1
        elif key_press == ord("r"):  # reset image
            self.img = copy(self.img_copy)
            self.drawing = False

        if cv.waitKey(10) & 0xFF == 27:
            return -1

        return 0

In [12]:
def main(path: str):
    main_obj = data_and_events(path)
    cv.namedWindow("image")
    cv.setMouseCallback("image", main_obj.mouse_callback)
    while True:
        img = main_obj.get_image()
        cv.imshow("image", img)
        if main_obj.keyboard_handler() == -1:
            break
    cv.destroyAllWindows()

In [ ]:
path = os.path.join(os.getcwd(), "..", "img", "lantern.jpg")
main(path)